# SBXCrossover

```java
    /**
     * Apply crossover to solution
     *
     * @param source the parents to operates.
     * @return the crossover elements
     * @throws NullPointerException     if source is null
     * @throws IllegalArgumentException if source size is other than 2
     */
    @Override
    public List<IntegerSolution> execute(List<IntegerSolution> source) {
        Objects.requireNonNull(source);
        if (source.size() != getNumberOfRequiredParents()) {
            throw new IllegalArgumentException("There must be two parents instead of " + source.size());
        }

        IntegerSolution parent1 = source.get(0);
        IntegerSolution parent2 = source.get(1);

        List<IntegerSolution> offspring = new ArrayList<IntegerSolution>(2);

        offspring.add((IntegerSolution) parent1.copy());
        offspring.add((IntegerSolution) parent2.copy());

        int i;
        double rand;
        double y1, y2, yL, yU;
        double c1, c2;
        double alpha, beta, betaq;
        int valueX1, valueX2;

        if (random.getRandomValue() <= crossoverProbability) {
            for (i = 0; i < parent1.getNumberOfVariables(); i++) {
                valueX1 = parent1.getVariable(i);
                valueX2 = parent2.getVariable(i);
                if (random.getRandomValue() <= 0.5) {
                    if (Math.abs(valueX1 - valueX2) > EPS) // EPS = 1.0e-14
                    {

                        if (valueX1 < valueX2) {
                            y1 = valueX1;
                            y2 = valueX2;
                        } else {
                            y1 = valueX2;
                            y2 = valueX1;
                        }

                        yL = parent1.getLowerBound(i);
                        yU = parent1.getUpperBound(i);
                        rand = random.getRandomValue();
                        //--------------------------------------
                        beta = 1.0 + (2.0 * (y1 - yL) / (y2 - y1));
                        alpha = 2.0 - Math.pow(beta, -(distributionIndex + 1.0));

                        if (rand <= (1.0 / alpha)) {
                            betaq = Math.pow((rand * alpha), (1.0 / (distributionIndex + 1.0)));
                        } else {
                            betaq = Math.pow(1.0 / (2.0 - rand * alpha), 1.0 / (distributionIndex + 1.0));
                        }

                        c1 = 0.5 * ((y1 + y2) - betaq * (y2 - y1));

                        //--------------------------------------
                        beta = 1.0 + (2.0 * (yU - y2) / (y2 - y1));
                        alpha = 2.0 - Math.pow(beta, -(distributionIndex + 1.0));

                        if (rand <= (1.0 / alpha)) {
                            betaq = Math.pow((rand * alpha), (1.0 / (distributionIndex + 1.0)));
                        } else {
                            betaq = Math.pow(1.0 / (2.0 - rand * alpha), 1.0 / (distributionIndex + 1.0));
                        }

                        c2 = 0.5 * (y1 + y2 + betaq * (y2 - y1));

                        if (c1 < yL) {
                            c1 = yL;
                        }

                        if (c2 < yL) {
                            c2 = yL;
                        }

                        if (c1 > yU) {
                            c1 = yU;
                        }

                        if (c2 > yU) {
                            c2 = yU;
                        }

                        if (random.getRandomValue() <= 0.5) {
                            offspring.get(0).setVariable(i, (int) c2);
                            offspring.get(1).setVariable(i, (int) c1);
                        } else {
                            offspring.get(0).setVariable(i, (int) c1);
                            offspring.get(1).setVariable(i, (int) c2);
                        }
                    } else {
                        offspring.get(0).setVariable(i, valueX1);
                        offspring.get(1).setVariable(i, valueX2);
                    }
                } else {
                    offspring.get(0).setVariable(i, valueX2);
                    offspring.get(1).setVariable(i, valueX1);
                }
            }
        }

        return offspring;
    }

```

Definición de valores

$y_1:$ variable de decision de una de las soluciones.

$y_2:$ variable de desicion de la otra solucion.

Se cumple que $y_1 < y_2$. Por lo tanto, a $y_1$ se le asigna la variable que tiene el menor valor y a $y_2$ la otra.

$yL:$ Lower Bound

$yU:$ Upper Bound

$r:$ Random value con $0 \leq r \leq 1$

$d_i:$ distribution index

In [101]:
y1 = 3
y2 = 4
yL = 1
yU = 6
r = 0.5795277000586205
di = 20

# Valida
if (y1 > y2): raise ValueError("y1({y1}) tiene que ser menor que y2({y2})")
if ((r < 0) or (r > 1)): raise ValueError("r({r}) tiene que estar entre 0 y 1")
if (not(yL < yU)):  raise ValueError("yL({yL}) tiene que ser menor que yU({yU})")
if ((y1 < yL) or (y1 > yU)):  raise ValueError(f"y({y1}) tiene que  estar entre yL({yL}) y yU({yU})")
if ((y2 < yL) or (y2 > yU)):  raise ValueError(f"y({y2}) tiene que  estar entre yL({yL}) y yU({yU})")

## Calculando C1

$$\beta = 1 + 2\frac{y_1-yL}{y_2-y_1}$$

$$\alpha = 2 - \frac{1}{\beta^{di+1}}$$

In [102]:
beta = 1 + 2*((y1-yL)/(y2-y1))
beta

5.0

In [103]:
alpha = 2 - 1/(beta**(di+1))
alpha

1.999999999999998

 
$$
\begin{cases} 
\beta_q=\sqrt[d_i+1]{r\alpha} & \text{si $r\leq \frac{1}{\alpha}$} \\ 
\beta_q=\sqrt[d_i+1]{\frac{1}{2-r\alpha}} &  \text{si $r > \frac{1}{\alpha}$}
\end{cases}
$$


In [104]:
def getBetaQ(alpha, r, di):
    if (r <= 1/alpha):
        return (r*alpha)**(1/(di+1))
    else:
        return (1/(2-r*alpha))**(1/(di+1))

$$C_1 = 0.5 ((y_1+y_2)-\beta_q(y_2-y_1))$$

In [105]:
bq= getBetaQ(alpha,r,di)
bq

1.008283140525676

In [106]:
c1 = 0.5*((y1+y2)-bq*(y2-y1))
c1

2.995858429737162

## Calculando C2

$$\beta = 1 + 2 \cdot \frac{yU-y_2}{y_2-y_1}$$

$$\alpha = 2 - \frac{1}{\beta^{di+1}}$$

In [107]:
beta = 1 + 2 *((yU-y2)/(y2-y1))
beta

5.0

In [108]:
alpha = 2 - 1/(beta**(di+1))
alpha

1.999999999999998

$$C_2 = 0.5 ((y_1+y_2)+\beta_q(y_2-y_1))$$

In [109]:
bq= getBetaQ(alpha,r,di)
bq

1.008283140525676

In [110]:
c2 = 0.5*((y1+y2)+bq*(y2-y1))
c2

4.0041415702628385

## Resultados 

In [111]:
print("c1 = ", int(c1))
print("c2 = ", int(c2))

c1 =  2
c2 =  4


# Polynomial Mutation 

``` java
    /**
     * Perform the mutation operation
     */
    private void doMutation(double probability, IntegerSolution solution) {
        double rnd, delta1, delta2, mutPow, deltaq;
        double y, yL, yU, val, xy;

        for (int i = 0; i < solution.getNumberOfVariables(); i++) {
            if (random.getRandomValue() <= probability) {
                y = (double) solution.getVariable(i);
                yL = solution.getLowerBound(i);
                yU = solution.getUpperBound(i);
                if (yL == yU) {
                    y = yL;
                } else {
                    delta1 = (y - yL) / (yU - yL);
                    delta2 = (yU - y) / (yU - yL);
                    rnd = random.getRandomValue();
                    mutPow = 1.0 / (distributionIndex + 1.0);
                    if (rnd <= 0.5) {
                        xy = 1.0 - delta1;
                        val = 2.0 * rnd + (1.0 - 2.0 * rnd) * (Math.pow(xy, distributionIndex + 1.0));
                        deltaq = Math.pow(val, mutPow) - 1.0;
                    } else {
                        xy = 1.0 - delta2;
                        val = 2.0 * (1.0 - rnd) + 2.0 * (rnd - 0.5) * (Math.pow(xy, distributionIndex + 1.0));
                        deltaq = 1.0 - Math.pow(val, mutPow);
                    }
                    y = y + deltaq * (yU - yL);
                    y = repairSolutionVariableValue(y, yL, yU);
                }
                solution.setVariable(i, (int) y);
            }
        }
    }
```

Definición de valores

$y:$ variable de decision

$yL:$ Lower Bound

$yU:$ Upper Bound

$r:$ Random value con $0 \leq r \leq 1$

$d_i:$ distribution index

In [6]:
y = 1 # 6
yL = 1
yU = 6
r = 0.6182457396478912 # 0.5182380048259613
di = 20

# Valida
if ((r < 0) or (r > 1)): raise ValueError("r({r}) tiene que estar entre 0 y 1")
if (not(yL < yU)):  raise ValueError("yL({yL}) tiene que ser menor que yU({yU})")
if ((y < yL) or (y > yU)):  raise ValueError(f"y({y}) tiene que  estar entre yL({yL}) y yU({yU})")

## Calculando los $\Delta$'s

$$\Delta_1 = \frac{y-yL}{yU-yL}$$
$$\Delta_2 = \frac{yU-y}{yU-yL}$$

In [7]:
delta1 = (y-yL)/(yU-yL)
delta1

0.0

In [8]:
delta2 = (yU-y)/(yU-yL)
delta2

1.0

## Calculando $\Delta_Q$
$$
\begin{cases} 
\Delta_q = \sqrt[di+1]{2r+(1-2r)(1-\Delta_{1})^{di+1}} - 1 & \text{si $r\leq 0.5$} \\ 
\Delta_q = 1 - \sqrt[di+1]{2(1-r)+2(r-0.5)(1-\Delta_{2})^{di+1}} &  \text{si $r > 0.5$}
\end{cases}
$$

In [9]:
def getDeltaQ(delta1, delta2, r ,di):
    if (r<=0.5):
        return (2*r+(1-2*r)*(1-delta1)**(di+1))**(1/(di+1)) - 1
    else:
        return 1 - (2*(1-r)+2*(r-0.5)*(1-delta2)**(di+1))**(1/(di+1))

In [10]:
deltaq= getDeltaQ(delta1, delta2, r, di)
deltaq

0.012766897777053976

## Obteniendo el resultado
$$y = y + \Delta_q(yU-yL)$$

In [11]:
y = y + deltaq * (yU-yL)
y

1.06383448888527

In [12]:
y = int(y)
y

1